# SETTING UP CONNECTION TO SQL SERVER WITH DATASETS

In [1]:
import pandas as pd
import sqlite3
from IPython.display import display

conn = sqlite3.connect("pharm_data.db")

#Fetching list of tables we have access to, for reference
cursor = conn.cursor()
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("ALL TABLES: ",)
for table in tables: print(table[0],end=", ")
print("\b\b")

ALL TABLES: 
BENEFICIARY_COST, GEOGRAPHIC_LOCATOR, PLAN_INFO, ZIP_STATE_COUNTY, NCPDP, FORMULARY_DATA, NDC_DATA, RXCUI_DATA, PNF_DATA, 


# REQUESTING USER'S ZIP TO COMPUTE PDP_REGION_CODE

In [2]:
#Asks user for their zip code, gets state, determines pdp_region_code based on state
user_zip = input("Enter your zipcode: ")
user_state = pd.read_sql("select COUNTY,STATENAME\
                          from ZIP_STATE_COUNTY\
                          where ZIPCODE is '" + user_zip + "'", conn)\
            ['STATENAME'].values[0].rstrip()

user_pdp_region_code = pd.read_sql("select distinct PDP_REGION_CODE\
                                    from GEOGRAPHIC_LOCATOR\
                                    where STATENAME is '"+user_state+"'",conn)\
                       ['PDP_REGION_CODE'].values[0]

Enter your zipcode: 07040


# REQUESTING DRUGS FROM USER, YIELDING RXCUIs

In [110]:
user_generics = {(input("Enter a generic drug name: ").upper(),\
                  input("Enter a desired strength: ")):None\
                 for i in range(int(input("Enter how many drugs you wish to search: ")))}
for drug,strength in user_generics:
    user_generics[(drug,strength)] = pd.read_sql("select * from RXCUI_DATA where upper(PSN) like '"+drug+"%' and strength like '"+strength+" %MG' and GENERIC_RXCUI is ''",conn).drop_duplicates(subset='SXDG_RXCUI',keep='last')['RXCUI'].values[0]
user_generics

Enter how many drugs you wish to search: 3
Enter a generic drug name: ALLOPURINOL
Enter a desired strength: 300
Enter a generic drug name: METFORMIN
Enter a desired strength: 500
Enter a generic drug name: METOPROLOL SUCCINATE
Enter a desired strength: 50


{('ALLOPURINOL', '300'): '197320',
 ('METFORMIN', '500'): '861007',
 ('METOPROLOL SUCCINATE', '50'): '866436'}

# GETTING DATA FOR FIRST DRUG

In [109]:
display(pd.read_sql("select \
                    PLAN_INFO.CONTRACT_NAME, \
                    PLAN_INFO.PLAN_NAME, \
                    PLAN_INFO.CONTRACT_ID, \
                    PLAN_INFO.PLAN_ID, \
                    PLAN_INFO.SEGMENT_ID, \
                    PLAN_INFO.FORMULARY_ID, \
                    PLAN_INFO.PREMIUM, \
                    PLAN_INFO.DEDUCTIBLE \
            from \
                FORMULARY_DATA join PLAN_INFO \
                on substr(FORMULARY_DATA.FORMULARY_ID, 4) = PLAN_INFO.FORMULARY_ID \
            where \
                  PLAN_INFO.PDP_REGION_CODE is '"+str(user_pdp_region_code)+"' and \
                  PLAN_INFO.CONTRACT_ID like 'S%'\
            order by PLAN_INFO.FORMULARY_ID",conn)\
.drop_duplicates(subset='FORMULARY_ID',keep='first')\
.reset_index(drop=True))

,CONTRACT_NAME,PLAN_NAME,CONTRACT_ID,PLAN_ID,SEGMENT_ID,FORMULARY_ID,PREMIUM,DEDUCTIBLE
0,UNITEDHEALTHCARE INSURANCE COMPANY,AARP MedicareRx Preferred (PDP),S5820,3,0,18039,86.6,0.0
1,UNITEDHEALTHCARE INS. CO. AND UNITEDHEALTHCARE NY,AARP MedicareRx Saver Plus (PDP),S5921,349,0,18040,33.2,405.0
2,"SYMPHONIX HEALTH INSURANCE, INC.",Symphonix Value Rx (PDP),S0522,4,0,18041,30.4,405.0
3,UNITEDHEALTHCARE INS. CO. AND UNITEDHEALTHCARE NY,AARP MedicareRx Walgreens (PDP),S5921,386,0,18043,26.8,405.0
4,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Saver (PDP),S5810,38,0,18056,34.9,325.0
5,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Select (PDP),S5810,278,0,18057,19.7,405.0
6,FIRST HEALTH LIFE & HEALTH INSURANCE COMPANY,First Health Part D Value Plus (PDP),S5768,127,0,18059,56.2,0.0
7,CIGNA HEALTH AND LIFE INSURANCE COMPANY,Cigna-HealthSpring Rx Secure (PDP),S5617,18,0,18082,74.8,405.0
8,CIGNA HEALTH AND LIFE INSURANCE COMPANY,Cigna-HealthSpring Rx Secure-Extra (PDP),S5617,249,0,18083,52.9,0.0
9,HORIZON INSURANCE COMPANY,Horizon Medicare Blue Rx Standard (PDP),S5993,1,0,18130,47.7,405.0


In [121]:
for key in user_generics:
    display(pd.read_sql(" ".join(("select \
                        PLAN_INFO.CONTRACT_NAME, \
                        PLAN_INFO.PLAN_NAME, \
                        PLAN_INFO.CONTRACT_ID, \
                        PLAN_INFO.PLAN_ID, \
                        PLAN_INFO.SEGMENT_ID, \
                        PLAN_INFO.FORMULARY_ID, \
                        PLAN_INFO.PREMIUM, \
                        PLAN_INFO.DEDUCTIBLE, \
                        FORMULARY_DATA.RXCUI, \
                        FORMULARY_DATA.PRIOR_AUTHORIZATION_YN, \
                        FORMULARY_DATA.TIER_LEVEL_VALUE, \
                        BENEFICIARY_COST.COST_AMT_PREF \
                from \
                    FORMULARY_DATA join PLAN_INFO \
                        on substr(FORMULARY_DATA.FORMULARY_ID, 4) = PLAN_INFO.FORMULARY_ID \
                    join BENEFICIARY_COST \
                        on PLAN_INFO.CONTRACT_ID = BENEFICIARY_COST.CONTRACT_ID and \
                           PLAN_INFO.PLAN_ID = BENEFICIARY_COST.PLAN_ID and \
                           cast(FORMULARY_DATA.TIER_LEVEL_VALUE as integer) = \
                                                    cast(BENEFICIARY_COST.TIER as integer) \
                where \
                      PLAN_INFO.PDP_REGION_CODE is '"+str(user_pdp_region_code)+"' and \
                      FORMULARY_DATA.RXCUI is "+user_generics[key]+" and \
                      PLAN_INFO.CONTRACT_ID like 'S%'\
                order by PLAN_INFO.FORMULARY_ID").split()),conn)\
    .drop_duplicates(subset='FORMULARY_ID',keep='first')\
    .reset_index(drop=True))

,CONTRACT_NAME,PLAN_NAME,CONTRACT_ID,PLAN_ID,SEGMENT_ID,FORMULARY_ID,PREMIUM,DEDUCTIBLE,RXCUI,PRIOR_AUTHORIZATION_YN,TIER_LEVEL_VALUE,COST_AMT_PREF
0,UNITEDHEALTHCARE INSURANCE COMPANY,AARP MedicareRx Preferred (PDP),S5820,3,0,18039,86.6,0.0,197320,N,1,0.05
1,UNITEDHEALTHCARE INS. CO. AND UNITEDHEALTHCARE NY,AARP MedicareRx Saver Plus (PDP),S5921,349,0,18040,33.2,405.0,197320,N,1,0.05
2,"SYMPHONIX HEALTH INSURANCE, INC.",Symphonix Value Rx (PDP),S0522,4,0,18041,30.4,405.0,197320,N,1,0.05
3,UNITEDHEALTHCARE INS. CO. AND UNITEDHEALTHCARE NY,AARP MedicareRx Walgreens (PDP),S5921,386,0,18043,26.8,405.0,197320,N,1,0.00
4,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Saver (PDP),S5810,38,0,18056,34.9,325.0,197320,N,1,0.05
5,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Select (PDP),S5810,278,0,18057,19.7,405.0,197320,N,1,0.00
6,FIRST HEALTH LIFE & HEALTH INSURANCE COMPANY,First Health Part D Value Plus (PDP),S5768,127,0,18059,56.2,0.0,197320,N,1,0.05
7,CIGNA HEALTH AND LIFE INSURANCE COMPANY,Cigna-HealthSpring Rx Secure (PDP),S5617,18,0,18082,74.8,405.0,197320,N,1,0.05
8,CIGNA HEALTH AND LIFE INSURANCE COMPANY,Cigna-HealthSpring Rx Secure-Extra (PDP),S5617,249,0,18083,52.9,0.0,197320,N,1,0.05
9,HORIZON INSURANCE COMPANY,Horizon Medicare Blue Rx Standard (PDP),S5993,1,0,18130,47.7,405.0,197320,N,2,0.05


,CONTRACT_NAME,PLAN_NAME,CONTRACT_ID,PLAN_ID,SEGMENT_ID,FORMULARY_ID,PREMIUM,DEDUCTIBLE,RXCUI,PRIOR_AUTHORIZATION_YN,TIER_LEVEL_VALUE,COST_AMT_PREF
0,UNITEDHEALTHCARE INSURANCE COMPANY,AARP MedicareRx Preferred (PDP),S5820,3,0,18039,86.6,0.0,861007,N,1,0.05
1,UNITEDHEALTHCARE INS. CO. AND UNITEDHEALTHCARE NY,AARP MedicareRx Saver Plus (PDP),S5921,349,0,18040,33.2,405.0,861007,N,1,0.05
2,"SYMPHONIX HEALTH INSURANCE, INC.",Symphonix Value Rx (PDP),S0522,4,0,18041,30.4,405.0,861007,N,1,0.05
3,UNITEDHEALTHCARE INS. CO. AND UNITEDHEALTHCARE NY,AARP MedicareRx Walgreens (PDP),S5921,386,0,18043,26.8,405.0,861007,N,1,0.00
4,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Saver (PDP),S5810,38,0,18056,34.9,325.0,861007,N,1,0.05
5,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Select (PDP),S5810,278,0,18057,19.7,405.0,861007,N,1,0.00
6,FIRST HEALTH LIFE & HEALTH INSURANCE COMPANY,First Health Part D Value Plus (PDP),S5768,127,0,18059,56.2,0.0,861007,N,1,0.05
7,CIGNA HEALTH AND LIFE INSURANCE COMPANY,Cigna-HealthSpring Rx Secure (PDP),S5617,18,0,18082,74.8,405.0,861007,N,1,0.05
8,CIGNA HEALTH AND LIFE INSURANCE COMPANY,Cigna-HealthSpring Rx Secure-Extra (PDP),S5617,249,0,18083,52.9,0.0,861007,N,1,0.05
9,HORIZON INSURANCE COMPANY,Horizon Medicare Blue Rx Standard (PDP),S5993,1,0,18130,47.7,405.0,861007,N,2,0.05


,CONTRACT_NAME,PLAN_NAME,CONTRACT_ID,PLAN_ID,SEGMENT_ID,FORMULARY_ID,PREMIUM,DEDUCTIBLE,RXCUI,PRIOR_AUTHORIZATION_YN,TIER_LEVEL_VALUE,COST_AMT_PREF
0,UNITEDHEALTHCARE INSURANCE COMPANY,AARP MedicareRx Preferred (PDP),S5820,3,0,18039,86.6,0.0,866436,N,2,0.05
1,UNITEDHEALTHCARE INS. CO. AND UNITEDHEALTHCARE NY,AARP MedicareRx Saver Plus (PDP),S5921,349,0,18040,33.2,405.0,866436,N,1,0.05
2,"SYMPHONIX HEALTH INSURANCE, INC.",Symphonix Value Rx (PDP),S0522,4,0,18041,30.4,405.0,866436,N,1,0.05
3,UNITEDHEALTHCARE INS. CO. AND UNITEDHEALTHCARE NY,AARP MedicareRx Walgreens (PDP),S5921,386,0,18043,26.8,405.0,866436,N,1,0.00
4,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Saver (PDP),S5810,38,0,18056,34.9,325.0,866436,N,2,0.05
5,AETNA LIFE INSURANCE COMPANY,Aetna Medicare Rx Select (PDP),S5810,278,0,18057,19.7,405.0,866436,N,2,0.05
6,FIRST HEALTH LIFE & HEALTH INSURANCE COMPANY,First Health Part D Value Plus (PDP),S5768,127,0,18059,56.2,0.0,866436,N,2,0.05
7,CIGNA HEALTH AND LIFE INSURANCE COMPANY,Cigna-HealthSpring Rx Secure (PDP),S5617,18,0,18082,74.8,405.0,866436,N,1,0.05
8,CIGNA HEALTH AND LIFE INSURANCE COMPANY,Cigna-HealthSpring Rx Secure-Extra (PDP),S5617,249,0,18083,52.9,0.0,866436,N,1,0.05
9,HORIZON INSURANCE COMPANY,Horizon Medicare Blue Rx Standard (PDP),S5993,1,0,18130,47.7,405.0,866436,N,2,0.05


In [111]:
pd.read_sql("select * from FORMULARY_DATA limit 10",conn)

,FORMULARY_ID,FORMULARY_VERSION,CONTRACT_YEAR,RXCUI,NDC,TIER_LEVEL_VALUE,QUANTITY_LIMIT_YN,QUANTITY_LIMIT_AMOUNT,QUANTITY_LIMIT_DAYS,PRIOR_AUTHORIZATION_YN,STEP_THERAPY_YN
0,FORMULARY_ID,FORMULARY_VERSION,CONTRACT_YEAR,RXCUI,NDC,TIER_LEVEL_VALUE,QUANTITY_LIMIT_YN,QUANTITY_LIMIT_AMOUNT,QUANTITY_LIMIT_DAYS,PRIOR_AUTHORIZATION_YN,STEP_THERAPY_YN
1,00018000,9,2018,1551300,00002143380,2,Y,2,28,N,N
2,00018000,9,2018,1551306,00002143480,2,Y,2,28,N,N
3,00018000,9,2018,1745108,00002144511,2,N,,,Y,N
4,00018000,9,2018,855820,00002512130,2,Y,30,30,N,N
5,00018000,9,2018,855816,00002512330,2,Y,30,30,N,N
6,00018000,9,2018,207834,00002733511,2,N,,,Y,N
7,00018000,9,2018,1728075,00002762301,2,N,,,N,N
8,00018000,9,2018,1718936,00002763511,2,Y,2,28,N,N
9,00018000,9,2018,1657777,00002766901,2,N,,,Y,N
